# Voting Sample
This sample will guide throught the whole process of implementing, deploying and calling contract in tezos. Not all steps are elaborated in detail. For more detail focus on step and tezos commands, please consult [calculator sample](calculator_dapp.ipynb).

As a prerequisite it's required to have [source build of tezos and liquidity](../setup/source_install.ipynb).

## Preparation of environment
In this sample, we will use both alphanet and sandbox. Usage of environment can be configured using following commands.

For more info on sandbox mode, please consult [tezos documentation on sandbox](http://tezos.gitlab.io/mainnet/introduction/various.html#use-sandboxed-mode) or check [calculator dapp](calculator-dapp.ipynb)

In [1]:
export TEZOS_HOME=~/tezos-dev/tezos

#During the sample we will reference the variable to configure, if the contract should go to alphanet or sandbox.
export USE_ALPHANET=no

if [ "$USE_ALPHANET" = "yes" ]; then
    export TEZOS_HOST=127.0.0.1
    export TEZOS_PORT=8731
else 
    export TEZOS_HOST=127.0.0.1
    export TEZOS_PORT=18731
fi
export TEZOS_NODE_URL="$TEZOS_HOST:$TEZOS_PORT"


# This variable will disable disclaimer about used tezos network. For this guide we will 
# disable it to make command outputs less cluttered. 
export TEZOS_CLIENT_UNSAFE_DISABLE_DISCLAIMER=yes

For alphanet we will create alias, for sandbox we will start node and initialize client.

In [2]:
if [ "$USE_ALPHANET" = "yes" ]; then
    alias tezos-client="$TEZOS_HOME/tezos-client --addr $TEZOS_HOST --port $TEZOS_PORT"
else
    cd $TEZOS_HOME
    ./src/bin_node/tezos-sandboxed-node.sh 1 --connections 1  1>tezos_sandbox_log.txt 2>&1 &
    export NODE_PID=$!
    echo $NODE_PID > tezos_sandbox_pid.txt
    sleep 5
    
    #Initialization of the client
    eval `./src/bin_client/tezos-init-sandboxed-client.sh 1`
fi

[1] 80793
## Tezos address added: tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx
## Tezos address added: tz1gjaF81ZRRvdzjobyfVNsAeSC6PScjfQwN
## Tezos address added: tz1faswCTDciRzE4oJ9jn2Vm2dvjeyA9fUzU
## Tezos address added: tz1b7tUupMgCNw2cCLpKTkSD1NZzB5TkP2sv
## Tezos address added: tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv
## Tezos address added: tz1TGu6TN5GSez2ndXXeDX6LgUDvLzPLqgYV
## 
## The client is now properly initialized. In the rest of this shell
## session, you might now run `tezos-client` to communicate with a
## tezos node launched with `launch-sandboxed-node 1`. For instance:
## 
##   tezos-client rpc get /chains/main/blocks/head/metadata
## 
## Note: if the current protocol version, as reported by the previous
## command, is "Ps6mwMrF2ER2s51cp9yYpjDcuzQjsc2yAz8bQsRgdaRxw4Fk95H", you
## may have to activate in your "sandboxed network" the same economic
## protocol than used by the alphanet by running:
## 
##   tezos-activate-alpha
## 
## Warning: all the client data will be removed w

In [3]:
if [ "$USE_ALPHANET" != "yes" ]; then
    tezos-activate-alpha
    tezos-client bake for bootstrap1
fi


Injected BLwFk7m6GW2X
Injected block BLR6rg4ahCsU


### Initialization of account
**Tezos sandbox is comming with bootstrap accounts, that have sufficient amount of tez required for testing. In alphanet it's required to get the wallet with tez loaded from [faucet](https://faucet.tzalpha.net).**

This will provide a wallet in the form of a JSON file tz1__xxxxxxxxx__.json. To continue with tutorial, please download the wallet and place the file into `tz1__*.json` into `~/tezos-dev` folder. For sandbox we will transfer some tez from bootstrap accounts.

In [4]:
if [ "$USE_ALPHANET" = "yes" ]; then
    tezos-client activate account test_account with $TEZOS_HOME/../tz1__*.json
else
    tezos-client gen keys test_account
    tezos-client transfer 1000 from bootstrap5 to test_account --burn-cap 1 > output.txt &
    export PROCESS_PID=$!
    tezos-client bake for bootstrap1
    tezos-client bake for bootstrap1
    wait $PROCESS_PID && cat output.txt
fi

[2] 80841
Injected block BMDfYaD6DFXD
[2]+  Done                    tezos-client transfer 1000 from bootstrap5 to test_account --burn-cap 1 > output.txt
Node is bootstrapped, ready for injecting operations.
Estimated gas: 10100 units (will add 100 for safety)
Estimated storage: 257 bytes added (will add 20 for safety)
Operation successfully injected in the node.
Operation hash: oodLXgG5JEiwzAytfeXbFzoAUFeEwRrL2vNBNuj9NgWrH61Qsym
Waiting for the operation to be included...
Operation found in block: BLJxZMpakdpzGNGhZuFEcWGMoew4yx3NRRQ3P4t2ekSsWHW5HTL (pass: 3, offset: 0)
This sequence of operations was run:
  Manager signed operations:
    From: tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv
    Fee to the baker: ꜩ0.001275
    Expected counter: 1
    Gas limit: 10200
    Storage limit: 277 bytes
    Balance updates:
      tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv ........... -ꜩ0.001275
      fees(tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx,0) ... +ꜩ0.001275
    Transaction:
      Amount: ꜩ1000
      From: tz1

Afterwards we can verify balance on test_account.

In [5]:
echo "Balance for test_account: $(tezos-client get balance for test_account)"

Balance for test_account: 1000 ꜩ


During the guide we are using aliases registered within `tezos-client` for convenience reasons. It's possible to use also account hash/address as well. Following two commands are refering to the same account. 

## Development of Smart Contract

Liquidity is a language heavily inspired by the OCaml language. Before starting working with Liquidity we advise 
to check some OCaml, F# or ReasonML tutorials to familiarize with language syntax (eg. https://try.ocamlpro.com).

To learn more about Liquidity, refer to [Liquidity website](http://www.liquidity-lang.org).

### Sample Voting Contract
```ocaml
[%%version 0.4]
  
type storage = {
  candidates : (string, int) map;
  voters : (address, bool) map;
}

(* Initially we will provide names of candidates *)
let%init init_candidates (candidate_names : string list) =
  let candidates = List.fold (fun (elt, map) -> Map.add elt 0 map) candidate_names
    (Map[] : (string, int) map) in
  { candidates = candidates; voters = (Map : (address, bool) map) }

(* During vote user can select one candidate. His address is stored in map voters and vote is *)
(* incremented in map candidates *)
let%entry main (candidate_name : string) (storage : storage) =
  let addr = Current.source() in
  let storage =
    storage.voters <- match Map.find addr storage.voters with
    | None -> Map.add addr true storage.voters
    | Some x -> failwith ("Voter has already voted", addr)
  in

  let storage =
    storage.candidates <- match Map.find candidate_name storage.candidates with
    | None -> failwith("Candidate is not valid", candidate_name)
    | Some x -> Map.add candidate_name (x + 1) storage.candidates ;
  in
  (([] : operation list), storage)
```

In [6]:
mkdir -p ~/tezos-dev/voting-dapp
cd ~/tezos-dev/voting-dapp

/bin/cat <<EOM >Voting.liq
[%%version 0.4]

type storage = {
  candidates : (string, int) map;
  voters : (address, bool) map;
}

let%init init_candidates (candidate_names : string list) =
  let candidates = List.fold (fun (elt, map) -> Map.add elt 0 map) candidate_names
    (Map[] : (string, int) map) in
  { candidates = candidates; voters = (Map : (address, bool) map) }

let%entry main (candidate_name : string) (storage : storage) =
  let addr = Current.source() in
  let storage =
    storage.voters <- match Map.find addr storage.voters with
    | None -> Map.add addr true storage.voters
    | Some x -> failwith ("Voter has already voted", addr)
  in

  let storage =
    storage.candidates <- match Map.find candidate_name storage.candidates with
    | None -> failwith("Candidate is not valid", candidate_name)
    | Some x -> Map.add candidate_name (x + 1) storage.candidates ;
  in
  (([] : operation list), storage)
EOM



### Contract Simulation and Deployment
#### Running Simulation of Contract using Liquidity
Tezos node can execute the script in Michelson and return result even without storage to blockchain. This can be used to simulate contract call during testing and development.

Liquidity can handle transpilation to Michelson and correct call of the script on our behalf. 

During the simulation, it's required to prepare storage structure before calling the contract entry point.
This can be achieved using following command.

In [7]:
liquidity --tezos-node $TEZOS_NODE_URL \
 Voting.liq \
 --init-storage '["Anna"; "Peter"; "Bob"]'

Main contract Voting
Voting.liq:18.6-18.12: Warning: unused variable "x"
Evaluated initial storage: { candidates = (Map [("Peter", 0); ("Bob", 0); ("Anna", 0)]); voters = Map }
Constant initial storage generated in "voting.liq.init.tz"


This command will call storage function (function where declaration starts with let%init) and pass arguments after --init-storage parameter. Result of the call is evaluated storage as it will be stored to blockchain, but still in Liquidity. Resulting storage in Michelson format will be stored in *.init.tz file.

Liquidity storage:

    Evaluated initial storage: *1*

Michelson storage:

In [8]:
cat voting.liq.init.tz

(Pair { Elt  "Anna" 0 ; Elt  "Bob" 0 ; Elt  "Peter" 0} {})

Now we have all required to run contract simulation using liquidity. Following command will execute liquidity script using tezos node.

In [10]:
# in --run add 1 1 first parameter is name of entry point, 
# second represents input to the entry point, 
# third parameter represents storage
liquidity --tezos-node $TEZOS_NODE_URL \
 Voting.liq \
 --run main '"Anna"' '{ candidates = (Map [("Peter", 0); ("Bob", 0); ("Anna", 0)]); voters = Map }'


Main contract Voting
Voting.liq:18.6-18.12: Warning: unused variable "x"
{
  candidates = (Map [("Peter", 0); ("Bob", 0); ("Anna", 1)]);
  voters = (Map [(KT1BEqzn5Wx8uJrZNvuS9DVHmLvG9td3fDLi, true)])
}
# Internal operations: 0


In [11]:
#Voting second time should fail.
liquidity --tezos-node $TEZOS_NODE_URL \
 Voting.liq \
 --run main '"Anna"' '{
  candidates = (Map [("Peter", 0); ("Bob", 0); ("Anna", 1)]);
  voters = (Map [(KT1BEqzn5Wx8uJrZNvuS9DVHmLvG9td3fDLi, true)])
}'



Main contract Voting
Voting.liq:18.6-18.12: Warning: unused variable "x"
Voting.liq:18.16-18.58: Runtime error: in /chains/main/blocks/head/helpers/scripts/run_code
- proto.003-PsddFKi3.scriptRejectedRuntimeError


: 1

### Running of Contract Using Tezos Client
For this section we will use `liquidity` as a compiler only. When calling the command with the liquidity program, it will compile the code into the Michelson program. 


#### Compilation of Contract to Michelson
Liquidity is hiding lot of complexity, however in some scenarios it is required to use Michelson. Following sample is showing how to use liquidity in order to create michelson script.

When liquidity script is compiled, storage initialization and contract entry points are provided separately. 

In [17]:
liquidity --tezos-node $TEZOS_NODE_URL \
 Voting.liq
 
echo ------

cat Voting.liq.tz

echo ------

cat Voting.liq.initializer.tz

Main contract Voting
Voting.liq:18.6-18.12: Warning: unused variable "x"
Storage initializer generated in "./voting.liq.initializer.tz"
File "./voting.liq.tz" generated
If tezos is compiled, you may want to typecheck with:
  tezos-client typecheck script ./voting.liq.tz
------
parameter string;
storage (pair :storage (map %candidates string int) (map %voters address bool));
code { DUP ;
       DIP { CDR @storage_slash_1 } ;
       CAR @candidate_name_slash_2 ;
       SOURCE @addr ;
       DUUUP @storage ;
       CAR %candidates ;
       DUUUUP @storage ;
       CDR %voters ;
       DUUUP @addr ;
       GET ;
       IF_NONE
         { DUUUUP @storage ;
           CDR %voters ;
           PUSH bool True ;
           DUUUUP @addr ;
           DIP { SOME } ;
           UPDATE }
         { DUUUP @addr ; PUSH string "Voter has already voted" ; PAIR ; FAILWITH } ;
       SWAP ;
       PAIR @storage %candidates %voters ;
       DUP @storage ;
       CDR %voters ;
       DUUP @storage ;
       

Liquidity is providing compilation also for method invocations. This way it simplifies creation of required michelson statements.

In [16]:
# Evaluation of the storage structure - calling of storage function and resulting storage structure is outputed
liquidity --tezos-node $TEZOS_NODE_URL \
 Voting.liq \
 --init-storage '["Anna"; "Bob"; "Jenny"]'

echo ------
echo vote Anna
# Compilation of function call - calling of add entry point with argument "1"
liquidity --tezos-node $TEZOS_NODE_URL \
 Voting.liq \
 --data main '"Anna"'

Main contract Voting
Voting.liq:18.6-18.12: Warning: unused variable "x"
Evaluated initial storage: { candidates = (Map [("Jenny", 0); ("Bob", 0); ("Anna", 0)]); voters = Map }
Constant initial storage generated in "voting.liq.init.tz"
------
vote Anna
Main contract Voting
"Anna"


Now it's possible to simulate the contract directly in Michelson.

In [27]:
echo "Calling vote Anna, initial storage ["Anna"; "Bob"; "Jenny"]"
tezos-client run script voting.liq.tz on storage "$(<voting.liq.init.tz)" and input '"Anna"'

echo "Calling vote Anna again, reusing initial storage from previous call" 
tezos-client run script voting.liq.tz \
  on storage \
  '(Pair { Elt "Anna" 1 ; Elt "Bob" 0 ; Elt "Jenny" 0 } { Elt "KT1BEqzn5Wx8uJrZNvuS9DVHmLvG9td3fDLi" True })' \
  and input '"Anna"'


Calling vote Anna, initial storage [Anna; Bob; Jenny]
storage
  (Pair { Elt "Anna" 1 ; Elt "Bob" 0 ; Elt "Jenny" 0 }
        { Elt "KT1BEqzn5Wx8uJrZNvuS9DVHmLvG9td3fDLi" True })
emitted operations
  


Calling vote Anna again, reusing initial storage from previous call
Runtime error in contract KT1BEqzn5Wx8uJrZNvuS9DVHmLvG9td3fDLi:
  01: parameter string;
  02: storage (pair :storage (map %candidates string int) (map %voters address bool));
  03: code { DUP ;
  04:        DIP { CDR @storage_slash_1 } ;
  05:        CAR @candidate_name_slash_2 ;
  06:        SOURCE @addr ;
  07:        DUUUP @storage ;
  08:        CAR %candidates ;
  09:        DUUUUP @storage ;
  10:        CDR %voters ;
  11:        DUUUP @addr ;
  12:        GET ;
  13:        IF_NONE
  14:          { DUUUUP @storage ;
  15:            CDR %voters ;
  16:            PUSH bool True ;
  17:            DUUUUP @addr ;
  18:            DIP { SOME } ;
  19:            UPDATE }
  20:          { DUUUP @addr ; PUSH string "V

: 1

#### Deployment of Contract using Tezos Client
For deployment we need to create originated account. This can be achieved using `tezos-client originate contract...` call. During the origination it's required to provide our private/secret key ([command to retrieve](#Testing-account-creation)).

In [39]:
# We are creating new originated contract and registring it's alias Calculator with our tezos-client.
tezos-client originate contract Voting1 \
 for test_account \
 transferring 1 from test_account \
 running voting.liq.tz \
 --init "$(<voting.liq.init.tz)" \
 --burn-cap 1.358 > contract_output.txt &

export PROCESS_PID=$!

[ "$USE_ALPHANET" != "yes" ] && tezos-client bake for bootstrap1

wait $PROCESS_PID
cat contract_output.txt

[2] 81460
Injected block BLUWcj1fdeHN
[2]+  Done                    tezos-client originate contract Voting1 for test_account transferring 1 from test_account running voting.liq.tz --init "$(<voting.liq.init.tz)" --burn-cap 1.358 > contract_output.txt
Node is bootstrapped, ready for injecting operations.
Estimated gas: 35661 units (will add 100 for safety)
Estimated storage: 1358 bytes added (will add 20 for safety)
Operation successfully injected in the node.
Operation hash: opSBa2T9iq7cZytERrshzfVrb898VShZbqj8LK2KFwZxm5zPJYh
Waiting for the operation to be included...
Operation found in block: BLUWcj1fdeHNfvTi6qdBLk5ttAAHmPSRFyFCFkK7brgAdmnKVWv (pass: 3, offset: 0)
This sequence of operations was run:
  Manager signed operations:
    From: tz1iH2DD3JRBRFsXUvMPNQ4HVKMoGwyBsAmm
    Fee to the baker: ꜩ0.004933
    Expected counter: 5
    Gas limit: 35761
    Storage limit: 1378 bytes
    Balance updates:
      tz1iH2DD3JRBRFsXUvMPNQ4HVKMoGwyBsAmm ........... -ꜩ0.004933
      fees(tz1KqTp

Operation is not reflected in tezos blockchain until new block is baked. 

`tezos-client` registers alias for newly created contract. The alias is matching hash outputed in the originate call above.


In [32]:
CONTRACT_HASH=$(grep "New contract KT" contract_output.txt | cut -d" " -f3)
echo "Contract Hash $CONTRACT_HASH"
echo ------
echo "Voting listed among aliases"
tezos-client list known contracts

Contract Hash KT1PMFwJyYrisEaR9WXAw6MqqUFTkB6Bzuqj
------
Calculator listed among aliases
Voting: KT1PMFwJyYrisEaR9WXAw6MqqUFTkB6Bzuqj
test_account: tz1iH2DD3JRBRFsXUvMPNQ4HVKMoGwyBsAmm
activator: tz1TGu6TN5GSez2ndXXeDX6LgUDvLzPLqgYV
bootstrap5: tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv
bootstrap4: tz1b7tUupMgCNw2cCLpKTkSD1NZzB5TkP2sv
bootstrap3: tz1faswCTDciRzE4oJ9jn2Vm2dvjeyA9fUzU
bootstrap2: tz1gjaF81ZRRvdzjobyfVNsAeSC6PScjfQwN
bootstrap1: tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx


### Call of contract using tezos-client
Calling of smart contract is the same as transfering balance. When calling smart contract we are providing additional arguments, that should be in Michelson - refer to [Michelson compilation](#Compilation-of-Contract-to-Michelson) for sample how to get to Michelson function call for liquidity function.

In [34]:
#Calling our contract, entry point add (using parameter generated by liquidity compiled function call)
tezos-client \
 transfer 0 from test_account to $CONTRACT_HASH --arg '"Anna"' --burn-cap 1 > output.txt &
PROCESS_PID=$!

#Baking manually
[ "$USE_ALPHANET" != "yes"] && tezos-client bake for bootstrap1  #baking the block
wait $PROCESS_PID
cat output.txt

[2] 80966
Injected block BLxADxv3iw24
[2]+  Done                    tezos-client transfer 0 from test_account to $CONTRACT_HASH --arg '"Anna"' --burn-cap 1 > output.txt
Node is bootstrapped, ready for injecting operations.
Estimated gas: 31204 units (will add 100 for safety)
Estimated storage: 31 bytes added (will add 20 for safety)
Operation successfully injected in the node.
Operation hash: onmTiE2Ns7hxU3hQPCQSJreqATrAiBad7c1WqYAo7aiTEAP5uAB
Waiting for the operation to be included...
Operation found in block: BLxADxv3iw24ye5TrbpeaS75fzxZF2WQXXnQyDZZrmgGHm9jfST (pass: 3, offset: 0)
This sequence of operations was run:
  Manager signed operations:
    From: tz1iH2DD3JRBRFsXUvMPNQ4HVKMoGwyBsAmm
    Fee to the baker: ꜩ0.003395
    Expected counter: 4
    Gas limit: 31304
    Storage limit: 51 bytes
    Balance updates:
      tz1iH2DD3JRBRFsXUvMPNQ4HVKMoGwyBsAmm ........... -ꜩ0.003395
      fees(tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx,1) ... +ꜩ0.003395
    Transaction:
      Amount: ꜩ0
    

We can verify contract call by querying storage in blockchain directly.

In [35]:
tezos-client get script storage for Voting
# curl -s http://$TEZOS_NODE_URL/chains/main/blocks/head/context/contracts/$CONTRACT_HASH/storage | jq

Pair { Elt "Anna" 1 ; Elt "Bob" 0 ; Elt "Jenny" 0 }
     { Elt "tz1iH2DD3JRBRFsXUvMPNQ4HVKMoGwyBsAmm" True }


Alternatively we can check the block in which the operation was included.

In [36]:
curl -s http://$TEZOS_NODE_URL/chains/main/blocks/head | jq

{
  "protocol": "PsddFKi32cMJ2qPjf43Qv5GDWLDPZb3T3bF6fLKiF5HtvHNU7aP",
  "chain_id": "NetXgtSLGNJvNye",
  "hash": "BLxADxv3iw24ye5TrbpeaS75fzxZF2WQXXnQyDZZrmgGHm9jfST",
  "header": {
    "level": 10,
    "proto": 1,
    "predecessor": "BM4hjkTD52a85a2UZFYnMoTTrhF6DBkrSZ2KAjimU7uPkh26BXo",
    "timestamp": "2018-12-28T15:44:56Z",
    "validation_pass": 4,
    "operations_hash": "LLoae3UHQqL4fTKG4PHjWxaa3zApAV6SG7mjAs1AY4apdZY9K7865",
    "fitness": [
      "00",
      "000000000000000a"
    ],
    "context": "CoUi9YvevpP8nB8AN9UYPJwMuA8i8eZtGd5ErderZvoWUecP1YRD",
    "priority": 3,
    "proof_of_work_nonce": "0000000369097db7",
    "signature": "sigpuaMpF3VKUCtSrFZfSThcQNTph28aQnGkTSWVyzGHv3xxuP56XhaxiRRZR6EpM9tKtceKh8cGAVRzm7KkGw6KGMrnhm9V"
  },
  "metadata": {
    "protocol": "PsddFKi32cMJ2qPjf43Qv5GDWLDPZb3T3bF6fLKiF5HtvHNU7aP",
    "next_protocol": "PsddFKi32cMJ2qPjf43Qv5GDWLDPZb3T3bF6fLKiF5HtvHNU7aP",
    "test_chain_status": {
      "status": "not_running"
    },
    "max_operatio

                          },
                          {
                            "int": "1"
                          }
                        ]
                      },
                      {
                        "prim": "Elt",
                        "args": [
                          {
                            "string": "Bob"
                          },
                          {
                            "int": "0"
                          }
                        ]
                      },
                      {
                        "prim": "Elt",
                        "args": [
                          {
                            "string": "Jenny"
                          },
                          {
                            "int": "0"
                          }
                        ]
                      }
                    ],
                    [
                      {
                        "prim": "Elt",
                        "arg

## Implementation of application front-end
TBD

In [ ]:
tezos-client transfer 1000 from bootstrap1 to tz1hzTqcG8EqWGPM6pkNWs9Bd3FfwuWHE9iX

## Sandbox Clean-up

Command to terminate sandbox blockchain running in the background

In [ ]:
kill $(<$TEZOS_HOME/tezos_sandbox_pid.txt)